<a href="https://colab.research.google.com/github/OctoberFall/SoK-Security/blob/main/deepaid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using DeepAid for model interpretability 

Reference: DeepAID, Interpreting and Improving Deep Learning-Based Anomaly Detection in Security Applications [Github](https://github.com/dongtsi/DeepAID)

### Run each cell to execute the code. We will first download a google drive folder. The zipped folder contains all the required files for this notebook. LSTM_onehot is the pytorch model trained with DeepLog architecture. We used the open source implementation of [Link](https://github.com/wuyifan18/DeepLog)

https://drive.google.com/file/d/1mhraNt2Z8X6S6dFjjRN_hK3ph3sXPmjR/view?usp=sharing 

In [1]:
!gdown 1mhraNt2Z8X6S6dFjjRN_hK3ph3sXPmjR

Downloading...
From: https://drive.google.com/uc?id=1mhraNt2Z8X6S6dFjjRN_hK3ph3sXPmjR
To: /content/deepaid.zip
100% 336k/336k [00:00<00:00, 101MB/s]


In [2]:
!unzip "/content/deepaid.zip"

Archive:  /content/deepaid.zip
   creating: deepaid/
  inflating: deepaid/LSTM_onehot.pth.tar  
   creating: deepaid/__pycache__/
  inflating: deepaid/__pycache__/deeplog.cpython-38.pyc  
  inflating: deepaid/__pycache__/interpreter.cpython-38.pyc  
  inflating: deepaid/__pycache__/timeseries_onehot.cpython-38.pyc  
  inflating: deepaid/abnormal_data.npy  
  inflating: deepaid/deeplog.py      
  inflating: deepaid/interpreter.py  
  inflating: deepaid/log_key_meanning.npy  
  inflating: deepaid/timeseries_onehot.py  


In [3]:
import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve
import torch

def validate_by_rmse(rmse_vec,thres,label):
    pred = np.asarray([0] * len(rmse_vec))
    idx = np.where(rmse_vec>thres)
    pred[idx] = 1
    cnf_matrix = confusion_matrix(label, pred)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    TPR = (TP/(TP+FN))[1]
    FPR = (FP/(FP+TN))[1]
    print("TPR:",TPR,"|FPR:",FPR)

    return pred

class Normalizer:
    def __init__(self, 
            dim, 
            normer="minmax",
            online_minmax=False): # whether fit_transform online (see Kitsune), *available only for normer="minmax"

        self.dim = dim # feature dimensionality
        self.normer = normer
        if self.normer == 'minmax':
            self.online_minmax = online_minmax
            self.norm_max = [-np.Inf] * self.dim
            self.norm_min = [np.Inf] * self.dim
        else:
            raise NotImplementedError # Implement other Normalizer here
        
    def fit_transform(self,train_feat):
        if self.normer == 'minmax':
            return self._minmax_fit_transform(train_feat)
        else:
            raise NotImplementedError # Implement other Normalizer here

    def transform(self,feat):
        if self.normer == 'minmax':
            return self._minmax_transform(feat)
        else:
            raise NotImplementedError # Implement other Normalizer here

    def restore(self,feat):
        if self.normer == 'minmax':
            return self._minmax_restore(feat)
        else:
            raise NotImplementedError # Implement other Normalizer here
        
    def _minmax_fit_transform(self,train_feat):
        if not self.online_minmax:
            self.norm_min = np.min(train_feat,axis=0)
            self.norm_max = np.max(train_feat,axis=0)
            norm_feat = (train_feat - self.norm_min) / (self.norm_max-self.norm_min+1e-10)
            return norm_feat
        else:
            norm_feat = []
            self.norm_max, self.norm_min = np.asarray(self.norm_max), np.asarray(self.norm_min)
            for i in range(len(train_feat)):
                x = train_feat[i]
                self.norm_max[x>self.norm_max] = x[x>self.norm_max]
                self.norm_min[x<self.norm_min] = x[x<self.norm_min]
                norm_feat.append((x - self.norm_min) / (self.norm_max-self.norm_min+1e-10))
            return np.asarray(norm_feat)

    def _minmax_transform(self, feat):
        norm_feat = (feat - self.norm_min) / (self.norm_max-self.norm_min+1e-10)
        return norm_feat

    def _minmax_restore(self, feat):
        denorm_feat = feat * (self.norm_max-self.norm_min+1e-10) + self.norm_min
        return denorm_feat
    
    # def _olminmax_fit_transform(self, train_feat):
    #     norm_feat = []
    #     for i in range(len(train_feat)):
    #         x = train_feat[i]
    #         self.norm_max[x>self.norm_max] = x[x>self.norm_max]
    #         self.norm_min[x<self.norm_min] = x[x<self.norm_min]
    #         norm_feat.append(x - self.norm_min) / (self.norm_max-self.norm_min+1e-10)
    #     return np.asarray(norm_feat)
    
    # def _olminmax_transform(self, feat):
    #     norm_feat = (feat - self.norm_min) / (self.norm_max-self.norm_min+1e-10)
    #     return norm_feat


""" Deeplog tools """
def deeplogtools_seqformat(model, abnormal_data, num_candidates, index=0):
    import keras.utils.np_utils as np_utils
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    X = abnormal_data.copy()
    y, X = X[:,-1], np_utils.to_categorical(X[:,:-1])
    Output = model(torch.from_numpy(X).type(torch.float).to(device))
    TP_idx = []
    for i in range(len(Output)):
        output = Output[i]
        label = y[i]
        predicted = torch.argsort(output)[-num_candidates:]
        if label not in predicted:
            TP_idx.append(i)
    seq_feat = np_utils.to_categorical(abnormal_data[TP_idx])
    feat = seq_feat[index]
    seq = torch.from_numpy(feat[:-1,:]).to(device)
    label = torch.tensor(np.argmax(feat[-1])).unsqueeze(0).to(device)
    return seq,label, abnormal_data[TP_idx][index]

""" Multi LSTM tools """
def multiLSTM_seqformat(test_feat, seq_len = 5, index=0):
    import more_itertools

    X_test = more_itertools.windowed(test_feat[:,:],n=seq_len,step=1)
    X_test = np.asarray(list(X_test))
    y_test = np.asarray(test_feat[seq_len-1:])

    # print("X_test:",X_test.shape,"y_test:",y_test.shape)
    i = index
    interp_feat = y_test[i]
    seq_feat = np.asarray([X_test[i]]) 
    # print("seq_feat:",seq_feat.shape,"interp_feat:",interp_feat.shape)

    return seq_feat, interp_feat

In [4]:
import sys
sys.path.insert(1,'/content/deepaid')

In [5]:
import torch
from deeplog import *

In [6]:
from deeplog import LSTM_onehot
import torch
model = torch.load('/content/deepaid/LSTM_onehot.pth.tar')

In [8]:
"""Step 2: Find an anomaly you are interested in"""
abnormal_data = np.load('/content/deepaid/abnormal_data.npy')
idx = 100
seq, label, anomaly_timeseries = deeplogtools_seqformat(model, abnormal_data, num_candidates=9, index=idx)

In [9]:
seq 

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
  

In [10]:
label

tensor([9], device='cuda:0')

In [11]:
anomaly_timeseries

array([ 4, 10,  9, 13,  6,  7, 10, 13,  6, 10,  9])

In [13]:
"""Step 3: Create a DeepAID Interpreter"""

from timeseries_onehot import UniTimeseriesAID
feature_desc = np.load('/content/deepaid/log_key_meanning.npy') # feature_description
my_interpreter = UniTimeseriesAID(model, feature_desc=feature_desc, class_num=28)

Successfully Initialize <Univariate Timeseries Interptreter> for Model <LSTM_onehot>


In [14]:
"""Step 4: Interpret your anomaly and show the result"""
interpretation = my_interpreter(seq, label)
my_interpreter.show_table(anomaly_timeseries, interpretation)

/content/deepaid/timeseries_onehot.py:61: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss_accuracy = Bound(self.bound_thres-Logit(out)[label[0]])



Visualize Interpretation (Table View)
+------+-------------------------------------+-------+------+-------------------------------------+
| Ano. |               Meaning               | Diff. | Ref. |               Meaning*              |
+------+-------------------------------------+-------+------+-------------------------------------+
|  4   |      Receiving blk* src&dest:*      |       |  4   |      Receiving blk* src&dest:*      |
|  10  |  PktResponder* for blk* terminating |       |  10  |  PktResponder* for blk* terminating |
|  9   |       PktResponder* Exception       |       |  9   |       PktResponder* Exception       |
|  13  |  Exception in receiveBlock for blk* |       |  13  |  Exception in receiveBlock for blk* |
|  6   |   writeBlock* received exception*   |       |  6   |   writeBlock* received exception*   |
|  7   | PktResponder* for blk* Interrupted. |       |  7   | PktResponder* for blk* Interrupted. |
|  10  |  PktResponder* for blk* terminating |       |  10  |

/content/deepaid/timeseries_onehot.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if torch.max(Logit(out)).cpu().data > self.pos_thres:
/content/deepaid/timeseries_onehot.py:77: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  IDX2.append(torch.argmax(Logit(out)).cpu().data.numpy().tolist())


## Second experiment

In [16]:
"""Step 2: Find an anomaly you are interested in"""
abnormal_data = np.load('/content/deepaid/abnormal_data.npy')
idx = 10
seq, label, anomaly_timeseries = deeplogtools_seqformat(model, abnormal_data, num_candidates=9, index=idx)

In [17]:
seq 

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
  

In [18]:
label

tensor([4], device='cuda:0')

In [19]:
anomaly_timeseries

array([10,  8, 25, 17, 24,  4, 25, 16, 24, 17,  4])

In [21]:
"""Step 3: Create a DeepAID Interpreter"""

from timeseries_onehot import UniTimeseriesAID
feature_desc = np.load('/content/deepaid/log_key_meanning.npy') # feature_description
my_interpreter = UniTimeseriesAID(model, feature_desc=feature_desc, class_num=28)

Successfully Initialize <Univariate Timeseries Interptreter> for Model <LSTM_onehot>


In [22]:
"""Step 4: Interpret your anomaly and show the result"""
interpretation = my_interpreter(seq, label)
my_interpreter.show_table(anomaly_timeseries, interpretation)


Visualize Interpretation (Table View)
+------+--------------------------------------+-------+------+--------------------------------------+
| Ano. |               Meaning                | Diff. | Ref. |               Meaning*               |
+------+--------------------------------------+-------+------+--------------------------------------+
|  10  |  PktResponder* for blk* terminating  |       |  10  |  PktResponder* for blk* terminating  |
|  8   |      Received blk* size* from*       |       |  8   |      Received blk* size* from*       |
|  25  |  NS.addStoredBlock: blkMap updated   |       |  25  |  NS.addStoredBlock: blkMap updated   |
|  17  | Starting thread to transfer blk* to* |       |  17  | Starting thread to transfer blk* to* |
|  24  |        ask* to replicate* to*        |       |  24  |        ask* to replicate* to*        |
|  4   |      Receiving blk* src&dest:*       |       |  4   |      Receiving blk* src&dest:*       |
|  25  |  NS.addStoredBlock: blkMap updated